In [ ]:
!pip install mistralai tqdm psycopg2-binary sqlalchemy python-dotenv pandas termcolor openpyxl --quiet

%load_ext autoreload
%autoreload 2

from database.getdata import query_data, get_connection, execute_query
from database.matching import find_matches, compare_names

import pandas as pd 
import os
from mistralai import Mistral
from dotenv import load_dotenv
import tqdm

import json
load_dotenv()

## LOAD DATASET Dataset5050_Cheffes de poste 2023.xlsx - LISTE_FINALE_FILMS_RETENUS.xlsx


In [ ]:
# read csv file data/Dataset5050_Cheffes de poste 2023.xlsx - LISTE_FINALE_FILMS_RETENUS.csv

df = pd.read_csv('../database/data/Dataset5050_Cheffes de poste 2023.xlsx - LISTE_FINALE_FILMS_RETENUS.csv')
df

In [ ]:
df['Femme_Real'].unique()


In [ ]:
df['Bonus_Parité'].unique()

In [ ]:
df_cp = query_data('SELECT * FROM inegalites_cinema.cheffes_postes')
df_cp


In [ ]:
df_film = query_data('SELECT * FROM inegalites_cinema.film')
df_film

## J'AJOUTE LES FILMS QUI NE SONT PAS DANS LA BASE DE DONNEES

In [ ]:
for ii, row in tqdm.tqdm(df.iterrows(), total=df.shape[0]):
    film_name = row['Titre']
    visa = row['Visa'] if pd.notna(row['Visa']) else None
    genre_film = row['Genre_Film'] if pd.notna(row['Genre_Film']) else None
    bonus_parite = True if row['Bonus_Parité'] == 'x' else False
    devis = row['Devis'] if pd.notna(row['Devis']) else None
    femme_real = True if row['Femme_Real'] == 'Oui' else False

    # find film in database
    matches = find_matches(film_name, df_film, 'nom_originel')

    if len(matches) == 0:
        # add row to dataframe df_film
        df_film = pd.concat([df_film, pd.DataFrame({'nom_originel': [film_name],
                                                    'n_visa_exploitation': [visa],
                                                    'genre': [genre_film],
                                                    'bonus_parite': [bonus_parite],
                                                    'devis': [devis],
                                                    'femme_realisatrice': [femme_real]
                                                    })], ignore_index=True)

df_film.drop(columns='id').to_sql('film', get_connection(),
                                 schema='inegalites_cinema', if_exists='append', index=False)

## J'AJOUTE LES CHEFFES DE POSTES QUI NE SONT PAS DANS LA BASE DE DONNEES

In [ ]:
for ii, row in tqdm.tqdm(df.iterrows(), total=df.shape[0]):
    cp_nom = row['Nom']
    cp_poste = row['Poste']
    cp_genre = row['Genre']

    matches = find_matches(cp_nom, df_cp, 'nom')

    # if matche get first name
    if len(matches) >= 1:
        cp_nom = matches.iloc[0]['nom']

    df_cp = pd.concat([df_cp, pd.DataFrame({
                                            'nom': [cp_nom],
                                            'type_de_poste': [cp_poste],
                                            'genre': [cp_genre],
                                            })], ignore_index=True)
df_cp = df_cp.drop_duplicates()
df_cp.drop(columns='id').to_sql('cheffes_postes', get_connection(),
                                 schema='inegalites_cinema', if_exists='append', index=False)

In [ ]:
# # join df with df_film and df_cp
# df_merged = df.merge(df_film, left_on='Titre', right_on='nom_originel', how='left')
# df_merged.rename(columns={'id': 'id_film'}, inplace=True)
# df_merged = df_merged.merge(df_cp, left_on='Nom', right_on='nom', how='left')
# df_merged.rename(columns={'id': 'id_cheffes_postes'}, inplace=True)
#
# df_merged[['id_film', 'id_cheffes_postes']].to_sql(
#     'film_cheffes_postes',
#     get_connection(),
#     schema='inegalites_cinema',
#     if_exists='replace',
#     index=False
# )